# Instagram Graph API call

## 1. Get Credential (w/ dotenv)

To protect credential info, you have to make `.env` file  
Here's some examples  

1. INSTAGRAM_ACCESS_TOKEN
2. INSTAGRAM_BUSINESS_ACCOUNT
3. FB_APP_SECRET_CODE
4. FB_CLIENT_ID

In [1]:
import requests
import json
import os
from pprint import pprint as pp
from dotenv import load_dotenv

# load .env
load_dotenv()
IG_ACCESS_TOKEN=os.environ.get("IG_ACCESS_TOKEN")
# FB_APP_SECRET_CODE=os.environ.get('FB_APP_SECRET_CODE')
IG_BUSINESS_ID=os.environ.get('IG_BUSINESS_ID')
IG_USER_NAME=os.environ.get('IG_USER_NAME')

def get_creds():
    """
    {'access_token': '',
    'graph_domain': 'https://graph.facebook.com/',
    'graph_version': 'v20.0',
    'instagram_account_id': '',
    'ig_username': '',
    'endpoint_base': 'https://graph.facebook.com/v20.0/'}

    """
    creds = dict() 
    creds['access_token'] = IG_ACCESS_TOKEN   # access token for use with all api calls
    # creds['client_secret'] = FB_APP_SECRET_CODE  # client secret from facebook app
    creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
    creds['graph_version'] = 'v20.0' # version of api we are hitting
    creds['instagram_account_id'] = IG_BUSINESS_ID # user business account id
    creds['ig_username'] = IG_USER_NAME
    # with domain and version
    creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint
    
    return creds

## 2. Create Media Object Response & Publish Media

### (Optional) Create Carousel post

Steps required for making a carousel post are.  
1. Upload every images or video you want to make a carousel of one by one and store the container_id in a list.  
2. Make another container that contains the container_id list.  
3. Publish the container.  

In [18]:
import requests

def create_media(params, include_caption=True, include_children=False):
    """Create media container object

    API Endpoint:
    - For image
        https://graph.facebook.com/v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
    - For video
        https://graph.facebook.com/v20.0/{ig-user-id}/media?media_type=VIDEO&video_url={video-url}&caption={caption}&access_token={access_token}
    - For Carousel
        https://graph.facebook.com/v20.0/{ig-user-id}/media?media_type=CAROUSEL&children={children}&caption={caption}&access_token={access_token}

    Returns:
        json object: {id : media_container_object_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    
    endpointParams = {
        'access_token': params['access_token']
    }

    ##### Media Type : IMAGE, VIDEO, CAROUSEL #####
    if params['media_type'] == 'IMAGE':
        endpointParams['image_url'] = params['media_url']
    
    elif params['media_type'] == 'VIDEO':
        endpointParams['video_url'] = params['media_url']
        endpointParams['media_type'] = params['media_type']
    
    # media_type == 'CAROUSEL'
    else:
        endpointParams['media_type'] = params['media_type']

    
    
    ##### Number of Media to Post : Single, Multi(=Carousel) #####
    
    # for single media post
    if include_caption:
        endpointParams['caption'] = params['caption']
        
    # for carousel.
    if include_children:
        endpointParams['children'] = params['children']
        response = requests.post(url, json=endpointParams)
        return response.json()

    response = requests.post(url, data=endpointParams)
    return response.json()

def create_single_media(params):
    """Create `single media` container object"""
    return create_media(params, include_caption=True)

def create_multi_media(params):
    """Create `multi media` container object"""
    return create_media(params, include_caption=False)

def create_slide_object(params):
    """Create `slide media` container object"""
    return create_media(params, include_caption=True, include_children=True)

def publish_media(media_object_id, params):
    """ Publish `media container` object
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media_publish?creation_id={creation-id}&access_token={access_token}

    Returns:
        json object: {id: media_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media_publish'

    endpointParams = dict()
    endpointParams['creation_id'] = media_object_id
    endpointParams['access_token'] = params['access_token']

    response = requests.post(url, data=endpointParams)
    
    print("\n---- MEDIA OBJECT PUBLISHED ---- \n")
    print("\tID:") # label
    print("\t" + response.json()['id']) # id of the object

    return response.json()


## 3. Post Comment & Reply for comment

- Use to write Q_Comment & A_reply

In [30]:
def post_comment(media_id, params):
    """Post comment to media
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_media_id}/comments?message={message}&access_token={access_token}


    """
    url= params['endpoint_base'] + media_id + '/comments'
    endpointParams= dict()
    endpointParams['message']= params['message']
    endpointParams['access_token']= params['access_token']
    
    response= requests.post(url, data= endpointParams)
    
    return response.json()

In [32]:
params['message']= 'welcome!'
post_response= post_comment('18081006226491173', params)
post_response

{'id': '18042057970935110'}

In [29]:
def post_reply_to_comment(comment_id, params):
    """ Post reply to a comment

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies?message={message}&access_token={access-token}

    Returns:
        Status Code <Response>
        - 200 : OK
        - 400 : Not supported input form
    """
    
    url = params['endpoint_base'] + comment_id + '/replies'
    
    endpointParams = dict()
    endpointParams['message'] = params['reply_message']
    endpointParams['access_token'] = params['access_token']
    response = requests.post(url, data=endpointParams)
    return response

In [33]:
params['reply_message'] = "I'll always happy to see you"
post_response = post_reply_to_comment(post_response['id'], params)
post_response

<Response [200]>

## 4. Answering to Question for the client with LLM (Reply)

### Reply Sequence
- 1. Get replies in comment
- 2. Check - Is it already replied?
  - If yes) Is the last response was created by media creator? 
- 3. Get all texts in comment (Optional - TBD)
- 4. Create reply response (w/ LLM)
- 5. Post reply to a comment

### 1) Get User Media & Comments

- get media id
- get comment in media

In [15]:
def get_media_id(params):
    """ Get User Media id
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}?fields=media&access_token={access_token}

    Returns:
        json object:
    
        {'media': {'data': [{'id': 'media_id_1'},
                            {'id': 'media_id_2'},
                            {'id': 'media_id_3'}],
                    'paging': {'cursors': {'before': '----',
                                            'after': '----'}
                                }
                    },
        'id': 'ig_business_id'}
    """
    
    url = params['endpoint_base'] + params['instagram_account_id'] + '?fields=media'
    
    endpointParams = {
        'access_token': params['access_token']
    }
    response = requests.get(url, endpointParams)
    return response.json()

def get_media_comments(media_id, params):
    """  Get media comments

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{media_id}/comments?access_token={access_token}
    
    Returns:
        json object:

        {'data' : [{'timestamp': {created_time},
                    'text': {comment_text_1},
                    'id' : {comment_id_1}},

                    {'timestamp': {created_time},
                    'text': {comment_text_2},
                    'id': {comment_id_2}}
                ]
        }
    """

    url = params['endpoint_base'] + media_id + '/comments?'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']

    response = requests.get(url, endpointParams)
    return response.json()


In [17]:
from pprint import pprint as pp
params= get_creds()
my_media = get_media_id(params)
pp(my_media)

{'id': '17841467459376742',
 'media': {'data': [{'id': '18045758038888861'},
                    {'id': '18242713690268915'},
                    {'id': '18015084977202592'},
                    {'id': '17934155543781257'},
                    {'id': '18081006226491173'},
                    {'id': '17875156758081829'},
                    {'id': '17880694839039025'},
                    {'id': '17923930097826286'},
                    {'id': '18141697960332903'},
                    {'id': '18261320584219788'},
                    {'id': '18031764533117210'},
                    {'id': '18246322663248696'},
                    {'id': '18070419385554044'},
                    {'id': '18326877682176492'},
                    {'id': '17891996034048780'},
                    {'id': '17890594719053636'},
                    {'id': '17855659533214875'},
                    {'id': '18045962785887885'}],
           'paging': {'cursors': {'after': 'QVFIUkpQWTlvNVRzTkcwLUFYWDlPcjlLTDZAyQlZA6eEN

In [18]:
# get all media comments
comments_data = []

for data in my_media['media']['data']:
    temp_media_id = data['id']
    comments_data.append(get_media_comments(temp_media_id, params))

pp(comments_data)

[{'data': [{'id': '18099769330426902',
            'text': 'yeah!',
            'timestamp': '2024-07-21T03:45:39+0000'},
           {'id': '18080360953496350',
            'text': '문제를 맞춰봅시다 ^&^',
            'timestamp': '2024-07-19T20:21:34+0000'}]},
 {'data': [{'id': '18014943341218752',
            'text': 'So cute',
            'timestamp': '2024-07-20T07:22:02+0000'}]},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': [{'id': '17846962674254771',
            'text': '맞팔해요 ^&^',
            'timestamp': '2024-07-19T04:57:46+0000'}]},
 {'data': [{'id': '17915879570957592',
            'text': '안녕하세요',
            'timestamp': '2024-07-18T10:31:38+0000'}]},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': []},
 {'data': [{'id': '17874199341067145',
            'text': '멋져요! 누구인가요?',
            'timestamp': '2024-07-18T04:54:02+0000'}]},
 {'data': [{'id': '17912313272967135',
            'text': '귀여운 고영희 !',
  

In [19]:
comments_id = []
comments= []

for media_comments in comments_data:
    for comment in media_comments['data']:
        comments_id.append(comment['id'])
        comments.append(comment['text'])
    
print(comments_id)
print(comments)


['18099769330426902', '18080360953496350', '18014943341218752', '17846962674254771', '17915879570957592', '17874199341067145', '17912313272967135']
['yeah!', '문제를 맞춰봅시다 ^&^', 'So cute', '맞팔해요 ^&^', '안녕하세요', '멋져요! 누구인가요?', '귀여운 고영희 !']


### 2) Is it already replied?

- get reply in comment
- is it from me?

In [21]:
def get_comment_replies(comment_id, params):
    """Create reply for comment
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies?fields={fields}&access_token={access-token}   ######확인
        v20.0/{ig_comment_id}/replies
    
    Returns:
        json object: the replies are `stacked-shaped`(LIFO)
        
        {'data': [{'id': {replied_id_2},
                'text': {replied_text_2}
                'username': {ig_username},

                {'id', {replied_id_1},
                'text': {replied_text_1},
                'username': {ig_username}
                ]}
    """

    url = params['endpoint_base'] + comment_id + '/replies'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'id, text, username'
    response = requests.get(url, endpointParams)
    return response.json()

In [22]:
# def is_replied(replies, params):
#     """Check if last response's username is creator's name

#     Returns:
#         True or False
#     """
#     if len(replies['data']):
#         return True if replies['data'][0]['username'] == params['ig_username'] else False

In [26]:
reply_response= get_comment_replies(comments_id[1], params)
reply_response

{'data': [{'id': '17869843359146531',
   'text': '@sesac.nalanhi 메렁',
   'username': 'sesac.nalanhi'},
  {'id': '17860720827142900',
   'text': '@sesac.nalanhi 쉴어효',
   'username': 'sesac.nalanhi'},
  {'id': '17993556578653501',
   'text': '정답을 알아봐요! \n\n O~X~',
   'username': 'sesac.nalanhi'}]}

In [66]:
is_replied(reply_response, params)

True

### 3) If not users(False), answer the question with LLM

- comment text & comment id
- Run Model (input : comment text)
- post reply with model_output for comment id

In [60]:
from openai import OpenAI
def get_reply_texts(comment):
    """
        comment를 입력으로 받아 이에 대한 답변을 LLM 모델로 생성
        여기서 무관한 comment시, 
        1) '모른다'라는 답변을 생성할 것인지,
        2) 아니면 reply하지 않을 것인지,
        3) 혹은 그냥 이모지 :) 생성할 것인지 (reply 확인용)
    """

    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "user", "content": comment}
        ]
    )
    
    return completion.choices[0].message.content

In [ ]:
# def reply_for_comment(params):
#     ##### get_media_id #####
#     url = params['endpoint_base'] + params['instagram_account_id'] + '?fields=media'
    
#     endpointParams = {
#         'access_token': params['access_token']
#     }
#     response = requests.get(url, endpointParams)
#     media_id_list= response.json()['media']['data']
    
    
#     ##### get_media_comments #####
#     for media in media_id_list:
#         url = params['endpoint_base'] + media['id'] + '/comments?'
#         response = requests.get(url, endpointParams)
#         comment_list_per_media= response.json()['data']   # timestamp, text, id(comment_id)
        
#         ##### get_comment_replies #####
#         for comment in comment_list_per_media:
#             url = params['endpoint_base'] + comment['id'] + '/replies'
#             endpointParams['fields'] = 'id, text, username'
#             response = requests.get(url, endpointParams)
            
#             ##### is_replied #####
#             reply_list_per_comment = response.json()['data']
            
#             # reply exist
#             if reply_list_per_comment:   
#                 i_replied= [bool(reply['username']==params['ig_username']) for reply in reply_list_per_comment]
                
#                 # All False (내가 한 reply가 없음)
#                 if not any(i_replied):   
#                     params['reply_message']= get_reply_texts()
#                     post_reply_to_comment(comment['id'],params)
            
#             # reply no exist
#             else:
#                 params['reply_message']= get_reply_texts()
#                 post_reply_to_comment(comment['id'],params)

In [ ]:
def reply_for_comment(params):
    ##### get_media_id #####
    media_id_list= get_media_id(params)['media']['data']    
    
    ##### get_media_comments #####
    for media in media_id_list:
        comment_list_per_media= get_media_comments(media['id'], params)['data']
        
        ##### get_comment_replies #####
        for comment in comment_list_per_media:
            reply_list_per_comment= get_comment_replies(comment['id'], params)['data']
            
            ##### is_replied #####            
            ## reply exist
            if reply_list_per_comment:   
                i_replied= [bool(reply['username']==params['ig_username']) for reply in reply_list_per_comment]
                
                # All False (내가 한 reply가 없음)
                if not any(i_replied):   
                    params['reply_message']= get_reply_texts(comment['text'])
                    post_reply_to_comment(comment['id'],params)
            
            ## reply no exist
            else:
                params['reply_message']= get_reply_texts(comment['text'])
                post_reply_to_comment(comment['id'],params)

## 5. Final Merge

최종 포스팅 함수

In [19]:
import pandas as pd

def post_media(data:pd.DataFrame):
    """ Run post process

    Sequence
        1. read_csv ('data.csv')
        2. post to instagram - with using 1st record
        3. return csv
            - len(data) >= 1 (pop first record and return remains)
            - len(data) == 0 (return blank data)
    
    Data Structure
    {
        'caption':  posting caption,
        'media_info': list of media url & media type
        'Q_comment': question comment
        'A_reply' : answer reply
    }
    """

    # data의 데이터가 있으면 진행
    if len(data):
        
        ##### Import Data & Upload #####
        posting_data = data.iloc[0]

        params = get_creds()
        params['caption'] = posting_data['caption']

        # single_media upload
        if (len(posting_data['media_info']) == 1):
            params['media_url'], params['media_type'] = posting_data['media_info'][0]
            media_container = create_single_media(params)

        # multi_media upload
        else:
            media_id_list = []

            # media container 생성 - 최대 10개까지만 업로드 가능
            for (media_url, media_type) in posting_data['media_info'][:10]: #################### 10개 미만이어도 오류 x?
                params['media_type'] = media_type
                params['media_url'] = media_url
                temp_media_container = create_multi_media(params)
                media_id_list.append(temp_media_container['id'])
                
            # carousel로 넘기기
            params['media_type'] = 'CAROUSEL'
            params['children'] = media_id_list
            media_container = create_slide_object(params)
        
        # published 된 경우 object published 출력
        post_id = publish_media(media_container['id'], params) 
        
        
        
        ##### post comment (Question) #####
        # 글 생성과 동시에 Question 생성하는 경우
        if posting_data['Q_comment']:
            params['message'] = posting_data['Q_comment']
            comment_id = post_comment(post_id['id'], params)

            # post reply to comment (Answer)
            # 글 생성과 동시에 Q&A 까지 함께 진행하는 경우
            if posting_data['A_reply']:
                params['reply_message'] = posting_data['A_reply']
                reply_id = post_reply_to_comment(comment_id['id'], params)###################################

        
        ##### Save Remaining Data #####
        # 작업했던 데이터(첫번째 행) 제거 후, 나머지 data를 csv로 저장
        data = data.drop(index=0).reset_index(drop=True)  # 0번 인덱스 드랍
        data.to_csv('data.csv', encoding='utf-8', index=False) # 값 저장.
        print(f'\tRemain records: {len(data)}')
        return 
    
    # data에 데이터가 없으면 바로 종료
    else:
        print("\n---- Sorry, Check the file plz :D ---- \n")
        print(f'\tNothing to post in `data.csv`')
        return


## 6. Test

### 샘플 인풋 데이터 생성 (`data.csv`)
- data['media_url'] 형식 (url, media_type) 으로 변경

In [ ]:
import pandas as pd

# Create sample data
df = pd.DataFrame(columns=['upload_time', 'media_info', 'caption', 'Q_comment', 'A_reply'])
info = [('https://img1.daumcdn.net/thumb/R1280x0.fjpg/?fname=http://t1.daumcdn.net/brunch/service/user/32E9/image/BA2Qyx3O2oTyEOsXe2ZtE8cRqGk.JPG', 'IMAGE'),('https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg', 'IMAGE')]
temp_str = '\n'*20

# carousel (multi-data)
data_multi = pd.DataFrame({
    'upload_time': ['2024/07/13'],
    'media_info': [info],
    'caption' : ['multi media test'],
    'Q_comment': ['문제를 맞춰봅시다 ^&^'],
    'A_reply' : [f'정답을 알아봐요! {temp_str} O~X~']})

# single media
single_info = [('https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg', 'IMAGE')]
data_single = pd.DataFrame({
    'upload_time': ['2024/07/20'],
    'media_info': [single_info], # (url, media_type)
    'caption' : ['single media test'],
    'Q_comment': ['no_comments'],
    'A_reply' : [None]})

data_df = pd.concat([data_multi, data_single], axis=0, ignore_index=True)
data_df.to_csv('data.csv', encoding='utf-8', index=False)


In [ ]:
pd.set_option('display.max_columns', None) ## 모든 열 출력
pd.set_option('display.max_rows', None) ## 모든 행 출력
pd.set_option('display.max_colwidth', None) ## 모든 내용 출력

data_multi['media_info']

0    [(https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg, IMAGE), (https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg, IMAGE)]
Name: media_info, dtype: object

### Test 시,

In [68]:
# 데이터 불러오기
import ast
data = pd.read_csv('data.csv', encoding='utf-8', converters={'media_info' : ast.literal_eval}) 
data

,upload_time,media_info,caption,Q_comment,A_reply
0,2024/07/13,"[(https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg, IMAGE), (https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg, IMAGE)]",multi media test,문제를 맞춰봅시다 ^&^,정답을 알아봐요! \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n O~X~
1,2024/07/20,"[(https://octapi.lxzin.com/interior/vImgFileSeq/202210/11/8ede80a1-1d0c-4839-bcc3-97bd4f357ecd.jpg, IMAGE)]",single media test,no_comments,NaN


In [69]:
# queue.csv를 읽은 후, df상태로 넣어주면 자동 처리.
post_media(data)


---- MEDIA OBJECT PUBLISHED ---- 

	ID:
	18045758038888861
	Remain records: 1


In [61]:
# 참고문서
"""
https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment

https://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급

https://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation""" # 댓글에 답글 작성


'\nhttps://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial\n\nhttps://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation'